In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

W dziesiejszym odcinku:
1. Wczytanie obrazka, wyświetlenie i konwersja do skali szarości
2. Przekształcenia morfologiczne - open/close/erozja/dylatacja/low-pass/high-pass
4. Dodawanie/odejmowanie/łączenie obrazków
5. Progowanie i binaryzacja
6. Detekcja krawędzi/linii
7. Case study: liczenie obiektów/twarzy na obrazku

In [ ]:
# Wczytanie obrazka i konwersja do skali szarości
image = cv2.imread('../samples/lena.png')
RGB_im = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
# wyświetlenie
plt.imshow(RGB_im)

In [ ]:
plt.imshow(gray, cmap='gray')

In [ ]:
# Funkcje pomocnicze

def morph(img) -> dict:
    """
        Funkcja do demonstracji przeksztalcen morfologicznych
    :param img: obraz wejsciowy
    :return: slownik z wyjsciowymi obrazami
    """
    kernel = np.ones((5,5),np.uint8)
    morph_ops = { 'original': img,
                 'erosion': cv2.erode(img, kernel, iterations=1),
                 'dilation': cv2.dilate(img, kernel, iterations=1),
                 'opening': cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel),
                 'closing': cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel),
                 'gradient': cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel),
                 'tophat': cv2.morphologyEx(img, cv2.MORPH_TOPHAT, kernel),
                 'blackhat': cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, kernel)}
    return morph_ops

def img_is_color(img):
    """
    Funkcja pomocnicza do sprawdzania przestrzeni kolorow obrazu
    :param img: obrz wejsciowy
    :return: True jesli obraz jest w skali szarosci
    """
    if len(img.shape) == 3:
        # Check the color channels to see if they're all the same.
        c1, c2, c3 = img[:, : , 0], img[:, :, 1], img[:, :, 2]
        if (c1 == c2).all() and (c2 == c3).all():
            return True

    return False

def show_image_list(images: dict, grid=True, num_cols=2, figsize=(20, 10), title_fontsize=30):
    """
    Funkcja pomocnicza do ladnego wyswietlania obrazow
    :param images:
    :param list_cmaps:
    :param grid:
    :param num_cols:
    :param figsize:
    :param title_fontsize:
    """
    num_images  = len(images)
    num_cols    = min(num_images, num_cols)
    num_rows    = int(num_images / num_cols) + (1 if num_images % num_cols != 0 else 0)

    # Create a grid of subplots.
    fig, axes = plt.subplots(num_rows, num_cols, figsize=figsize)

    # Create list of axes for easy iteration.
    if isinstance(axes, np.ndarray):
        list_axes = list(axes.flat)
    else:
        list_axes = [axes]
    counter = 0
    for label, imgg in images.items():
        cmap   = None if img_is_color(imgg) else 'gray'
        list_axes[counter].imshow(imgg, cmap=cmap)
        list_axes[counter].set_title(label, fontsize=title_fontsize)
        list_axes[counter].grid(grid)
        counter+=1

    for i in range(num_images, len(list_axes)):
        list_axes[i].set_visible(False)

    fig.tight_layout()
    _ = plt.show()

In [ ]:
# Przekształcenia morfologiczne - open/close/erozja/dylatacja/low-pass/high-pass
img = cv2.imread('../samples/j.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

show_image_list(morph(img),
                num_cols=3,
                figsize=(100, 100),
                grid=False,
                title_fontsize=50)

In [ ]:
# Dodawanie/odejmowanie/łączenie obrazków
circle = cv2.imread('../samples/circle.png')
circle = cv2.cvtColor(circle, cv2.COLOR_BGR2GRAY)
star = cv2.imread('../samples/star.png')
star = cv2.cvtColor(star, cv2.COLOR_BGR2GRAY)

combined = cv2.add(star, circle)
subtracted = cv2.subtract(star, circle)
show_image_list({'circle': circle, 'star': star, 'combined': combined, 'subtracted': subtracted},figsize=(10,10))

In [ ]:
# Binaryzacja / progowanie
img = cv2.imread('../samples/parrot.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
ret,thresh1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
_,thresh2 = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
_,thresh3 = cv2.threshold(img,127,255,cv2.THRESH_TRUNC)
_,thresh4 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO)
_,thresh5 = cv2.threshold(img,127,255,cv2.THRESH_TOZERO_INV)

show_image_list({'original': img, 'THRESH_BINARY': thresh1, 'THRESH_BINARY_INV': thresh2, 'THRESH_TRUNC': thresh3, 'THRESH_TOZERO': thresh4, 'THRESH_TOZERO_INV': thresh5},figsize=(10,10))


In [ ]:
# Progowanie adaptacyjne

img = cv2.imread('../samples/sudoku.PNG')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# img = cv2.medianBlur(img,5)
_,th1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,11,2)

show_image_list({'original': img, 'Global Thresholding (v = 127)': th1, 'Adaptive Mean Thresholding': th2, 'Adaptive Gaussian Thresholding': th3},figsize=(10,10))

In [ ]:
# Detekcja krawędzi/linii

img = cv2.imread('../samples/lena.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
# Convert to graycsale
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# Blur the image for better edge detection
img_blur = cv2.GaussianBlur(img_gray, (3,3), 0)

# Sobel Edge Detection
sobelx = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
sobely = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
sobelxy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection

# Canny Edge Detection
canny = cv2.Canny(image=img_blur, threshold1=100, threshold2=200) # Canny Edge Detection

show_image_list({'original': img, 'Gray': img_gray, 'Blured': img_blur,
                 'Sobel X': sobelx, 'Sobel Y': sobely, 'Sobel XY': sobelxy,
                 'Canny': canny
                 },
                figsize=(20,20)
                )


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 38)